In [ ]:
#r "nuget: TorchSharp-cpu"

using TorchSharp;
using static TorchSharp.torch;
using static TorchSharp.TensorExtensionMethods;
using static TorchSharp.torch.distributions;

# Models

While doing numerical computing with Torch is useful and productive, modeling is what Torch is all about, whether the host language is Python, C#, or F#. It's its reason for being.

In this tutorial, we will introduce the basics of modeling with TorchSharp. The intent is not to teach you how to come up with a good model, or do data science. It's merely to teach the mechanics of model construction and use in TorchSharp.

It's important to understand how LibTorch, the engine underneath Pytorch and TorchSharp works -- it's a __dynamic__ model engine, which means that the engine is there to keep track of work that is done from the host language, tracing it and doing the necessary bookeeping to be able to perform backpropagation. This is very different form how early frameworks worked, for example Tensorflow v1.X -- in those frameworks, a model graph was built by the host code, and only once the graph was built could you run the model.

The static approach of TF v1.X had some challenges associated with it, one of which was that it was hard to debug when code didn't run eagerly. One of the upsides of the static approach is that it's relatively straight-forward to externalize the whole model: its weights and its logic.

That is not so with a dynamic framework -- the weights and the logic are tied, but only loosely. In order to execute the model during training, and after, you need the code for the model, expressed in the host language. Thus, the focus of much of this discussion will be on the model code.

## Model Classes

Typically, we want to keep the logic of a model in a class of its own. This makes it easy to use other TorchSharp constructs to combine and run models. Conceptually, that's all a model is -- a Tensor -> Tensor function implemented as a class with a 'forward()' function. This function is where the logic of the model is placed. (If C# supported operator(), like C++ does, that's what we'd use here, instead. Just like Python does.)

TorchSharp makes it easy to build models, because you only have to specify the forward function. To do backprogation, you also need the backward function, which supports using the chain rule of calculus to calculate gradients. In Torch, the backward function is automatically implemented as long as the forward function relies only on Torch APIs for computations.

Let's start with a super-simple model, which has only one single linear layer that expect a tensor with 1000 input elements and will yield a tensor with 100 elements:

In [ ]:
private class Trivial : nn.Module<Tensor,Tensor>
{
    public Trivial()
        : base(nameof(Trivial))
    {
        RegisterComponents();
    }

    public override Tensor forward(Tensor input)
    {
        return lin1.forward(input);
    }

    private nn.Module<Tensor,Tensor> lin1 = nn.Linear(1000, 100);
}

We can now invoke the model with some random data:

In [ ]:
var input = rand(1000);
var model = new Trivial();
model.forward(input)

When we say that a model expects a tensor with 1000 inputs, we're actually ignoring that in most scenarios, data is fed into a model in batches. The size of the Linear input is actually just about the last dimension. We can feed tensors of Nx1000 and will get a tensor Nx100 out:

In [ ]:
model.forward(rand(3,1000))

It's perfectly possible to have more than one dimension preceding the last, but that is not very common. Batches are usually just presented by adding one first dimension.

In [ ]:
model.forward(rand(2,3,1000))

All TorchSharp operators will accept either batched or non-batched input data, but not all operators will accept more than one preceding dimension. Linear is not unique in this regard, but it's not generally the case.

If all we wanted to use was a single Linear layer, the model class wouldn't strictly speaking be necessary. The real value of the class, which matches most real-world scenarios, is to combine more than one operator in the model, and abstract (hide) it from the outside world. The most simple addition we can make is to add ReLU to the model.

Since ReLU, as do all activation functions, does not rely on trainable weights, there is no need to create a module and keep it in a field, we can just invoke it as a function. There is a module version of ReLU, too, which has its use, but in this situation, it's overkill.

In [ ]:
private class Trivial : nn.Module<Tensor,Tensor>
{
    public Trivial()
        : base(nameof(Trivial))
    {
        RegisterComponents();
    }

    public override Tensor forward(Tensor input)
    {
        using var x = lin1.forward(input);
        return nn.functional.relu(x);
    }

    private nn.Module<Tensor,Tensor> lin1 = nn.Linear(1000, 100);
}

With ReLU in place, there will be no negative numbers in the output, as expected.

In [ ]:
var input = rand(1000);
var model = new Trivial();
model.forward(input)

To make the model slightly more interesting, let's add another layer:

In [ ]:
private class Trivial : nn.Module<Tensor,Tensor>
{
    public Trivial()
        : base(nameof(Trivial))
    {
        RegisterComponents();
    }

    public override Tensor forward(Tensor input)
    {
        using var x = lin1.forward(input);
        using var y = nn.functional.relu(x);
        return lin2.forward(y);
    }

    private nn.Module<Tensor,Tensor> lin1 = nn.Linear(1000, 100);
    private nn.Module<Tensor,Tensor> lin2 = nn.Linear(100, 10);
}

In [ ]:
var model = new Trivial();
model.forward(input)

With this, we're ready to start training the model. Let's make up some fake (random) data, and then train the model.

In [ ]:
var dataBatch = rand(32,1000);  // Our pretend input data
var resultBatch = rand(32,10);  // Our pretend ground truth.
dataBatch.ToString()

First, we need a loss function that compares the output from the model with the ground truth, our labels. Here, we can use mean square error, a.k.a. "MSE". The loss produces a scalar value that represents how good the prediction is compared with the truth. Close to 0 is good, far from it is bad. It's a good idea to know how bad the loss is when we start, i.e. when the model is completely random.

Loss functions are modules, too, but their `forward()` methods usually take more than one input argument.

In [ ]:
var loss = nn.MSELoss();
loss.forward(model.forward(dataBatch), resultBatch).item<float>()

## Training

Training the consists of repeatedly computing the loss, then the gradients, then applying the gradient to the model weights before starting over.

In [ ]:
var learning_rate = 0.001f;

// Compute the loss
var output = loss.forward(model.forward(dataBatch), resultBatch);

// Clear the gradients before doing the back-propagation
model.zero_grad();

// Do back-progatation, which computes all the gradients.
output.backward();

// Adjust the weights using the gradients.
using (torch.no_grad()) {
    foreach (var param in model.parameters()) {
        var grad = param.grad();
        if (grad is not null) {
            var update = grad.mul(learning_rate);
            param.sub_(update);
        }
    }
}

loss.forward(model.forward(dataBatch), resultBatch).item<float>()


After training once, we should see a smaller loss. If you keep running the cell above over and over (Ctrl-Enter instead of Shift-Enter), the loss should decrease. Not by very much, perhaps, but it should keep going toward zero.

## Optimizers

The last step of the training logic above is called 'optimization,' which is used in its mathematical sense: we're minimizing the loss function, i.e. finding the inputs for which the loss is the smallest. Note that we are not minimizing it with respect to the input data, we're minimizing with respect to the weights.

Explicitly writing out the minimization steps like above will get very tedious, and far more complex optimization approaches exist than the simple on above (gradient descent). These are captured in classes appropriately called 'optimizers'. Instead of doing the above, we can use an optimizer and use it to adjust the weights.

In [ ]:
var learning_rate = 0.001f;

var optimizer = torch.optim.SGD(model.parameters(), learning_rate);

// Compute the loss
var output = loss.forward(model.forward(dataBatch), resultBatch);

// Clear the gradients before doing the back-propagation
model.zero_grad();

// Do back-progatation, which computes all the gradients.
output.backward();

optimizer.step();

loss.forward(model.forward(dataBatch), resultBatch).item<float>()

There are lots of optimizers available, and it's rarely the case that you can know beforehand which one will be good for your model. How do you measure what is good? 

What we care about in training is to quickly reach the point where further training is pointless, because the loss no longer goes down. You want to get there spending as little time and resources as possible. One thing you can do is adjust the learning rate, but you may have to choose a more sophisticated optimizer (which do more compute and add training time).

In this case, a bigger learning rate will make the loss go down quicker, but that isn't always the case. If the learning rate is too big, you can get in trouble, too. Among other things, the loss may never converge toward its lowest point.

In [ ]:
var learning_rate = 0.01f;

var optimizer = torch.optim.SGD(model.parameters(), learning_rate);

// Compute the loss
var output = loss.forward(model.forward(dataBatch), resultBatch);

// Clear the gradients before doing the back-propagation
model.zero_grad();

// Do back-progatation, which computes all the gradients.
output.backward();

optimizer.step();

loss.forward(model.forward(dataBatch), resultBatch).item<float>()

Using a different optimizer is easy, but not always better. Sometimes, as in this case, a simple optimizer is better.

In [ ]:
var optimizer = torch.optim.Adam(model.parameters());

// Compute the loss
var output = loss.forward(model.forward(dataBatch), resultBatch);

// Clear the gradients before doing the back-propagation
model.zero_grad();

// Do back-progatation, which computes all the gradients.
output.backward();

optimizer.step();

loss.forward(model.forward(dataBatch), resultBatch).item<float>()

Usually, you put the training logic in a loop, iterating for a number of "epochs". Play around with the loop counter and see what that does. Even a few thousand iterations is reasonable on a CPU for this simple model. When I ran this for 10,000 iterations, I got a loss that was very close to zero.

In [ ]:
var learning_rate = 0.01f;
model = new Trivial();

var optimizer = torch.optim.SGD(model.parameters(), learning_rate);

for (int i = 0; i < 1000; i++) {
    // Compute the loss
    using var output = loss.forward(model.forward(dataBatch), resultBatch);

    // Clear the gradients before doing the back-propagation
    model.zero_grad();

    // Do back-progatation, which computes all the gradients.
    output.backward();

    optimizer.step();
}

loss.forward(model.forward(dataBatch), resultBatch).item<float>()

## Accuracy

The loss is useful for optimization, but it's a relatively crude indication of how good the model actually is. In fact, it's largely useless for that. What we really care about is how accurate it is at making predictions. For that, we need to count the number of correct predictions and divide by the batch size.

To accomplish that, we have to have some idea of how to interpret the output of the model. In this case, each prediction is a ten-element tensor. How do we read that? Let's decide that we will interpret the model as identifying the index with the largest output value. TorchSharp has a function to do just that, which we can use to evaluate the model after it's been trained.

For the ground truth, here are the maximums:

In [ ]:
var refMax = resultBatch.argmax(1);
refMax

So, we want the output of the model to match this as much as possible. By visual inspection, it looks pretty good.

In [ ]:
var predMax = model.forward(dataBatch).argmax(1);
predMax

Let's measure how good this really is. When the batch is thousands or millions of records, we can't visually inspect. This logic does an element-wise comparison, then sums up all the results (true == 1, false == 0) and divides by the number of elements. You can see, that even with a low loss value, the accuracy isn't that impressive, at least not when you run the training loop for 1,000 iterations. Increase to 10,000 and the results are better. When I ran it, the accuracy went up to 1.0, i.e. perfect.

There's only so much we can do with random input data and labels; you can't expect that a model will always find interesting patterns in random data. When you have real data, it's different.

In [ ]:
(refMax == predMax).sum() / predMax.numel()

To demonstrate that the model isn't actually very good at anything except memorizing its training data, let's see what the accuracy is with a new batch of data.

In [ ]:
var x = rand(64,1000);  // Our pretend input data
var y = rand(64,10);    // Our pretend ground truth.

var yMax = y.argmax(1);
var pMax = model.forward(x).argmax(1);

(yMax == pMax).sum() / refMax.numel()

I saw between 3-31% accuracy when I ran this. That's not very impressive -- random guessing should yield 10% accuracy on average. Again, though, it's random data, so we can't expect it to be.

## Serializing the Weights

That is the model creation and training in a nutshell. Once you have the model, you use the same logic for inferencing, i.e. using the model to make predictions.

It is, however, extremely rare that you will be using the model in the same process as you use to train it. In fact, the only situation when you do that, is when you use the model to evaluate its accuracy. To use it after it's been trained, in another process, you have to store the weights on disk and then reload them in the new process.

TorchSharp makes it easy to store the weights.

In [ ]:
model.save("tutorial6.model.bin");

To load the weights in a new process, you have to have the model code, create an instance of it with random weights, and then load the saved weights. Let's show how loading the weights makes a difference to the model predictions. Make sure that you save a model with good accuracy -- training for 2500 iterations gave me about 94%.

In [ ]:
var model1 = new Trivial();
predMax = model1.forward(dataBatch).argmax(1);
(refMax == predMax).sum() / predMax.numel()

In [ ]:
model1.load("tutorial6.model.bin");
predMax = model1.forward(dataBatch).argmax(1);
(refMax == predMax).sum() / predMax.numel()

Because it's fairly ugly to first create the model, then load the weights, there's a static method in `nn.Module` that does both. It will work for models with parameterless constructors, only.

In [ ]:
Trivial model2 = (Trivial)nn.Module.Create<Trivial>("tutorial6.model.bin");
predMax = model2.forward(dataBatch).argmax(1);
(refMax == predMax).sum() / predMax.numel()

A more elegant solution may be to add a constructor that takes a file path to your model. If you do, __make absolutely sure__ that the call to `load` comes after the call to `RegisterComponents`.

In [ ]:
private class Trivial : nn.Module<Tensor,Tensor>
{
    public Trivial(String path) : this()
    {
        this.load(path);
    }

    public Trivial() : base(nameof(Trivial))
    {
        RegisterComponents();
    }

    public override Tensor forward(Tensor input)
    {
        using var x = lin1.forward(input);
        using var y = nn.functional.relu(x);
        return lin2.forward(y);
    }

    private nn.Module<Tensor,Tensor> lin1 = nn.Linear(1000, 100);
    private nn.Module<Tensor,Tensor> lin2 = nn.Linear(100, 10);
}

In [ ]:
var model3 = new Trivial("tutorial6.model.bin");
predMax = model3.forward(dataBatch).argmax(1);
(refMax == predMax).sum() / predMax.numel()

There's a relatively detailed article about model serialization at: [Saving and Loading TorchSharp Model Weights](https://github.com/dotnet/TorchSharp/blob/main/docfx/articles/saveload.md), but a couple of things bear repeating here:

1. The format used for saving and restoring TorchSharp models is __not__ the same as what Python uses for PyTorch models. The PyTorch format relies on Python pickling, which is a Python-specific serialization format, not compatible with .NET.

2. Saving a model means saving its weights only. The model logic is __not__ saved. In order to load the weights, you have to have a model that is exactly like the one that saved the weights. Copy the code, or put it in a separate library that can be loaded by the training and inference code alike.

3. You can save a Pytorch model's weights using specific support we added to TorchSharp (see the article mentioned earlier for details). That still means that you have to translate the Python model code into a .NET model that has the same exact structure.

4. You cannot save a TorchSharp model and load it in Pytorch.

In the future, we are planning to support extracting both the logic and weights, when possible, by relying on ONNX as a language-agnostic whole-model serialization format. ONNX export is already supported by PyTorch, with some limitations.

### Using Tensorbard

Tensorboard is a tool that was originally built for Tensorflow, but it is general enough that it may be used with a number of data sources, including PyTorch and TorchSharp. The TorchSharp support for TB is limited, but it is useful for logging scalars during training, for example by logging accuracy or loss at the end of each epoch.

To use Tensorboard, you should create a 'SummaryWriter' instance and use it for logging. If  don't specify a logging directory, one will be created in 'runs' under the current directory. To clear out files, you have to manually delete them using the OS user interface for doing so.

In [ ]:
var writer = torch.utils.tensorboard.SummaryWriter("runs/trivial", createRunName:true);

var model = new Trivial();

var optimizer = torch.optim.SGD(model.parameters(), learning_rate);

for (int i = 0; i < 50; i++) {
    for (int j = 0; j < 20; j++) {
        // Compute the loss
        using var output = loss.forward(model.forward(dataBatch), resultBatch);

        // Clear the gradients before doing the back-propagation
        model.zero_grad();

        // Do back-progatation, which computes all the gradients.
        output.backward();

        optimizer.step();
    }

    var l = loss.forward(model.forward(dataBatch), resultBatch).item<float>();

    writer.add_scalar("csharp/loss", l, i);
}


## Sequential

The prior sections have described the most general way of constructing a model, that is, by creating a class that abstracts the logic of the model and explicitly calls each layer's `forward` methos. While it's not too complicated to do so, it's a lot of "ceremony" to accomplish something very regular.

Fortunately, for models, or components of models, that simply pass one tensor from layer to layer, there's a class to handle it. It's called `Sequential` and is created by passing a sequence of tuples. The first element of the tuple is the name of the layer (required), and the second is the component.

The following model is equivalent to the Trivial model we've seen before. In fact, weights from Trivial can be loaded into it.

In [ ]:
var seq = nn.Sequential(("lin1", nn.Linear(1000, 100)), ("relu1", nn.ReLU()), ("lin2", nn.Linear(100, 10)));
seq.load("tutorial6.model.bin");
predMax = model3.forward(dataBatch).argmax(1);
(refMax == predMax).sum() / predMax.numel()

How can that be? Well, it's because save/load doesn't actually save and restore the model, it saves and restores the weights. All it cares about is that the layer's with weights have the same definition, and that they have the same name. In the Trivial case, the names were derived from the field names, and in the Sequential case, they are explicitly given. ReLU doesn't have any weights, so the fact that we did it differently doesn't factor it. The name of the relu layer can be anything, it just has to be something that isn't `null`.

About `ReLU` -- in the Sequential case, we implemented that with a layer? That's because Sequential requires that its arguments be subclasses of `Module`, so a function doesn't work.

There's another use for `Sequential`: often, for more complex models, the model structure is comprised of blocks of layers that repeat a number of times. To construct such models, it is convenient to hand off the execution to a `Sequential` instance, which takes care of the necessary looping for you.


One example is **ResNet**, which has a number of different configurations, made up of repeating blocks of layers. 

The ResNet constructor looks like this:

```C#
public ResNet(string name, Func<string, int,int,int,Module> block, int expansion, IList<int> num_blocks, int numClasses, Device device = null) : base(name)
{
    if (planes.Length != strides.Length) throw new ArgumentException("'planes' and 'strides' must have the same length.");

    var modules = new List<(string, Module)>();

    modules.Add(($"conv2d-first", Conv2d(3, 64, kernelSize: 3, stride: 1, padding: 1, bias: false)));
    modules.Add(($"bnrm2d-first", BatchNorm2d(64)));
    modules.Add(($"relu-first", ReLU(inPlace:true)));
    MakeLayer(modules, block, expansion, 64, num_blocks[0], 1);
    MakeLayer(modules, block, expansion, 128, num_blocks[1], 2);
    MakeLayer(modules, block, expansion, 256, num_blocks[2], 2);
    MakeLayer(modules, block, expansion, 512, num_blocks[3], 2);
    modules.Add(("avgpool", AvgPool2d(new long[] { 4, 4 })));
    modules.Add(("flatten", Flatten()));
    modules.Add(($"linear", Linear(512 * expansion, numClasses)));

    layers = Sequential(modules);

    RegisterComponents();
}
```

Note that the building blocks are constructed in a list of tuples, which is passed to the Sequential constructor when the model is complete. The forward method is super-simple:

```C#
public override Tensor forward(Tensor input)
{
    return layers.forward(input);
}
```

The complete code for it can be found in [Resnet.cs](https://github.com/dotnet/TorchSharpExamples/blob/main/src/CSharp/Models/ResNet.cs)